## Order Dependent Colley

This notebook contains an order dependent version of Colley’s method. If you want to use Colley’s method, but you want a team’s win against another team to mean something different at the beginning and end of the season, this is the notebook will do that for you. If you want to assume that teams change over the coarse of the season, this is a good notebook for you.

This notebook clue values wins, losses, and who those are against, while the Order Dependent Massey notebook is more concerned with point differential of results. This notebook does not allow you to add weights to games, though that can be done in a similar way to how it’s done in OrderDependentMassey.

-Grant Harkins

In [ ]:
#This cell just collects filepaths and filenames to read later
pathGames = '/FILEPATH/'
pathTeams = '/FILEPATH/'
gameFilename = '.txt'
teamFilename = '.txt'

In [ ]:
import pandas as pd

teamNames = pd.read_csv(pathTeams + teamFilename, header = None)
numTeams = len(teamNames)

In [ ]:
#column 0 of the game file is days since 1/1/000
#column 1 is YYYYMMDD
#column 2 is team 1 id
#column 3 is team 1 homefield (1= home, -1 = away, 0 = neutral
#column 4 team1 score
#column 5 team2 id
#column 6 is team 2 homefield
#column 7 is team 2 score

games = pd.read_csv(pathGames + gameFilename, header = None)
ThnumGames = len(games)

In [ ]:
#The Calculations

import numpy as np

K = 1 #Each game counts as having been played once. You can increase this value for faster covergence 
#of ratings, but it will converge with less information 

ratings = (1/2)*np.ones(numTeams)
wins = []
losses = []
oppRatings = []

for i in range(numTeams):
    wins.append(0)
    losses.append(0)
    oppRatings.append(0)

numberCorrectPredictions = 0

for i in range(numGames):
    team1ID = games.loc[i,2] - 1
    team1Score = games.loc[i,4]
    team1Loc = games.loc[i,3]

    team2ID = games.loc[i,5] - 1
    team2Score = games.loc[i,7]
    team2Loc = games.loc[i,6]

    currentDay = games.loc[i,0]

    if team1Score > team2Score and ratings[team1ID] > ratings[team2ID] and (i >= 150):
        numberCorrectPredictions += 1
    elif team2Score > team1Score and ratings[team2ID] > ratings[team1ID] and (i >= 150):
        numberCorrectPredictions += 1
    elif team1Score == team2Score and ratings[team1ID] == ratings[team2ID] and (i >= 150):
        numberCorrectPredictions += 1

    if team1Score > team2Score:
        wins[team1ID] += K
        losses[team2ID] += K
    elif team2Score > team1Score:
        wins[team2ID] += K
        losses[team1ID] += K

    oppRatings[team1ID] += K*ratings[team2ID]
    oppRatings[team2ID] += K*ratings[team1ID]

    ratings[team1ID] = (1 + (wins[team1ID] - losses[team1ID]) / 2 + oppRatings[team1ID]) / (2 + wins[team1ID] + losses[team1ID])
    ratings[team2ID] = (1 + (wins[team2ID] - losses[team2ID]) / 2 + oppRatings[team2ID]) / (2 + wins[team2ID] + losses[team2ID])

## Printing and Sorting the Rankings/Ratings

In [ ]:
iSort = np.argsort(-ratings)
k = 0

print('\n\n************** OD Colley Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
if k == 0:
    for i in range(numTeams):
        print(f'{i+1:4d}   {ratings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')
else:
    for i in range(k):
        print(f'{i+1:4d}   {ratings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return

print(f'Predictability: {numberCorrectPredictions/(numGames-150)*100:.2f}%')
#print(f'mean squared error is {mnSqErr}')

print(sum(ratings))